In [ ]:
import pandas as pd
import numpy as np
import dalmatian as dm
import pdb
import math
import TerraFunction as terra
import sys
sys.path.insert(0, '../ccle_processing/')
import CCLE_postp_function as ccle
import rpy2
%load_ext rpy2.ipython
from taigapy import TaigaClient
from IPython.display import Image,display
from IPython.core.display import HTML 
%load_ext autoreload
%autoreload 2

In [ ]:
tc = TaigaClient()
dm.__version__

## initializing data

In [ ]:
maleintervals = "gs://fc-secure-fd4c24cf-6bfd-410a-9bca-e02642da12f8/immediate/intervalsXY.list"
femaleintervals = "gs://fc-secure-fd4c24cf-6bfd-410a-9bca-e02642da12f8/immediate/intervalsXX.list"
sample_values_todrop = ["sj_diseases","sj_datasets","attr_age_at_diagnosis","attr_sex","attr_race","attr_ethnicity","attr_diagnosis"]
outpath = "sample.tsv"
spamplepath = "SAMPLE_INFO.txt"
bucketname= "gs://fc-secure-fd4c24cf-6bfd-410a-9bca-e02642da12f8/"
samplesetname = "WGS_CNV_stjude"
namespace="rmc-rnaseq"
workspace="St_Jude_RMS"

## uploading the required interval list of chromosomes for the PON

In [ ]:
! echo "chr1\nchr2\nchr3\nchr4\nchr5\nchr6\nchr7\nchr8\nchr9\nchr10\nchr11\nchr12\nchr13\nchr14\nchr15\nchr16\nchr17\nchr18\nchr19\nchr20\nchr21\nchr22\nchrX\nchrY" > intervalsXY.list
! echo "chr1\nchr2\nchr3\nchr4\nchr5\nchr6\nchr7\nchr8\nchr9\nchr10\nchr11\nchr12\nchr13\nchr14\nchr15\nchr16\nchr17\nchr18\nchr19\nchr20\nchr21\nchr22\nchrX\nchrX" > intervalsXX.list
! gsutil cp intervalsXY.list $maleintervals
! gsutil cp intervalsXX.list $femaleintervals

## loading and Displaying data

In [ ]:
newsample = pd.read_csv(spamplepath,sep='\t')
newsample

In [ ]:
#only keeping whole genome
for i,val in enumerate(newsample["file_path"]):
    if val.split('/')[-1].split('.')[1] != "WholeGenome" or val.split('/')[-2]!= "bam":
        newsample = newsample.drop(i)
    elif val.split('/')[1] != 'gs:':
        newsample["file_path"][i]="gs://fc-secure-fd4c24cf-6bfd-410a-9bca-e02642da12f8/immediate/bam_wg/"+newsample["file_path"][i].split('/')[-1]
newsample = newsample.reset_index(drop=True)
newsample = newsample.rename(index=str, columns={"sample_name": "sample_id", "subject_name": "participant_id","file_path":"WGS_bam"})
currfile=""
bai = ['']*int(newsample.shape[0])
#creating an array of bai and adding it to their coresponding bams
for i in newsample.index:
    currfile=newsample["WGS_bam"][i]
    if currfile.split('/')[-1].split('.')[-1] == "bai":
        bai[int(newsample[newsample["WGS_bam"]==currfile[:-4]].index.values[0])] = currfile
newsample["WGS_bam_index"]=pd.Series(bai,index=newsample.index)
#removing original bai rows
for i in newsample.index: 
    currfile=newsample["WGS_bam"][i]
    if currfile.split('/')[-1].split('.')[-1] == "bai":
        newsample = newsample.drop(i)
newsample = newsample.reset_index(drop=True)
newsample["sample_set"]=pd.Series([samplesetname]*int(newsample.shape[0]),index=newsample.index)
newsample.set_index("sample_id",inplace=True,drop=True)
newsample = newsample[newsample.columns.tolist()[1:]+[newsample.columns.tolist()[0]]]
newsample = newsample.loc[~newsample.index.duplicated(keep='first')]
newsample.to_csv(outpath,sep="\t")
newsample

Removing a corrupted sample

In [ ]:
newsample=newsample.drop("SJRHB007_G")
newsample=newsample.drop("SJRHB010_D")

## creating set for each samples

In [ ]:
wm = dm.WorkspaceManager(namespace, workspace)

samplesetname

In [ ]:
wm.upload_samples(newsample)
wm.update_sample_set(samplesetname,newsample.index)

## creating the sample set for normal cells with m/f

In [ ]:
normalsXX = newsample[(newsample["sample_type"]=="Germline") & (newsample["sj_datasets"]!="Clinical Pilot") &(newsample["attr_sex"]=="Female")].index.values
normalsXY = newsample[(newsample["sample_type"]=="Germline") & (newsample["sj_datasets"]!="Clinical Pilot") &(newsample["attr_sex"]=="Male")].index.values

In [ ]:
normalsXX

In [ ]:
wm.update_sample_set(samplesetname+"_normals_XY",normalsXY.tolist())

In [ ]:
wm.update_sample_set(samplesetname+"_normals_XX1",normalsXX.tolist())

## creating the sample set for cancer patients with/without germline cells sequenced

I am quite positive that the clinical pilot are not germline and have been misclassified

In [ ]:
# diagnosis samples
diagnosis = newsample[(newsample["sample_type"]!="Germline") | (newsample["sj_datasets"]=="Clinical Pilot")]['participant_id'].values.tolist()
# germline samples
germline = newsample[(newsample["sample_type"]=="Germline") & (newsample["sj_datasets"]!="Clinical Pilot")]['participant_id'].values.tolist()
# assert that we have them all in total
assert(len(diagnosis)+len(germline)==newsample.shape[0]) 
# people without a matching germline
patientswithout = set(diagnosis) - set(germline)
# samples from patients without a matching germline
toprocessPON = newsample[newsample["participant_id"].isin(patientswithout)].index.values.tolist()
# samples from patients with a matching germline
not_toprocessPON = newsample[~newsample["participant_id"].isin(patientswithout)][newsample["sample_type"]=="Germline"].index.values.tolist()

In [ ]:
wm.update_sample_set(samplesetname+"_not_toprocessPON",not_toprocessPON)
wm.update_sample_set(samplesetname+"_toprocessPON",toprocessPON)

In [ ]:
cnv_pon = wm.get_config("CNV_Somatic_Panel_Workflow")
cnv_pon

In [ ]:
cnv_pon['inputs']['CNVSomaticPanelWorkflow.normal_bams'] = 'this.samples.WGS_bam'
cnv_pon['inputs']['CNVSomaticPanelWorkflow.normal_bais'] = 'this.samples.WGS_bam_index'

## runing male PON
#a['inputs']['CNVSomaticPanelWorkflow.intervals'] = '"'+maleintervals+'"'


#wm.create_submission(a, setname+"_normals_XY", 'sample_set', use_callcache=True)

## running female PON
cnv_pon['inputs']['CNVSomaticPanelWorkflow.intervals'] = '"'+femaleintervals+'"'
wm.update_config(cnv_pon)
wm.create_submission(cnv_pon['name'], samplesetname+"_normals_XX1", 'sample_set', use_callcache=True)

## running specific PON

In [ ]:
pairs = newsample[~newsample["participant_id"].isin(patientswithout)][newsample["sample_type"]!="Germline"].index.values.tolist()
nomatched = newsample[newsample["participant_id"].isin(patientswithout)].index.values.tolist()

In [ ]:
sampleset = wm.get_sample_sets()
samples = wm.get_samples()
new_dataset = pd.DataFrame(columns=[samplesetname+"_id","read_count_pon","WGS_bam_tumor","WGS_bam_normal","WGS_bam_normal_index","WGS_bam_tumor_index","intervals"])

In [ ]:
samples=samples.drop(labels="SJRHB007_G")
samples=samples.drop(labels="SJRHB010_D")

In [ ]:
for val in nomatched:
    sex = "_normals_XY" if newsample["attr_sex"][val]!="Female" else "_normals_XX1" 
    intervals = maleintervals if samples["attr_sex"][val]!="Female" else femaleintervals 
    new_dataset = new_dataset.append({
        samplesetname+"_id": val,
        "read_count_pon": sampleset["read_count_pon"][samplesetname+sex],
        "WGS_bam_tumor": samples["WGS_bam"][val],
        "WGS_bam_normal": "",
        "WGS_bam_normal_index": "",
        "WGS_bam_tumor_index": samples["WGS_bam_index"][val],
        "intervals": intervals
    },ignore_index=True)

In [ ]:
for val in pairs:
    paired_germline = samples[samples["participant"]==samples["participant"][val]][samples["sample_type"]=="Germline"]
    sex = "_normals_XY" if samples["attr_sex"][val]!="Female" else "_normals_XX" 
    read_count_pon = sampleset["read_count_pon"][samplesetname+sex]
    intervals = maleintervals if samples["attr_sex"][val]!="Female" else femaleintervals 
    new_dataset = new_dataset.append({
        samplesetname+"_id": val,
        "read_count_pon": read_count_pon,
        "WGS_bam_tumor": samples["WGS_bam"][val],
        "WGS_bam_normal": paired_germline["WGS_bam"].values[0],
        "WGS_bam_normal_index": paired_germline["WGS_bam_index"].values[0],
        "WGS_bam_tumor_index": samples["WGS_bam_index"][val],
        "intervals": intervals
    },ignore_index=True)

In [ ]:
new_dataset.set_index(samplesetname+"_id",inplace=True,drop=True)

In [ ]:
wm.upload_entities(samplesetname,new_dataset)

In [ ]:
wm.update_sample_set(sample_ids=new_dataset.index.values.tolist(),sample_set_id=samplesetname)

In [ ]:
CNVpipeline = wm.get_config('CNV_Somatic_Pair_Workflow')
CNVpipeline

In [ ]:
CNVpipeline['inputs']['CNVSomaticPairWorkflow.tumor_bam'] = 'this.WGS_bam_tumor'
CNVpipeline['inputs']['CNVSomaticPairWorkflow.tumor_bam_idx'] = 'this.WGS_bam_tumor_index'
CNVpipeline['inputs']['CNVSomaticPairWorkflow.normal_bam'] = 'this.WGS_bam_normal'
CNVpipeline['inputs']['CNVSomaticPairWorkflow.normal_bam_idx'] = 'this.WGS_bam_normal_index'
CNVpipeline['inputs']['CNVSomaticPairWorkflow.padding'] = 250
CNVpipeline['rootEntityType']= 'pair'
CNVpipeline['inputs']['CNVSomaticPairWorkflow.common_sites'] = 'workspace.common_sites_hg38'
## runing male PON
CNVpipeline['inputs']['CNVSomaticPairWorkflow.intervals'] = 'workspace.interval_list'
CNVpipeline['inputs']['CNVSomaticPairWorkflow.read_count_pon'] = 'this.read_count_pon'
CNVpipeline['inputs']['CNVSomaticPairWorkflow.bin_length'] = 1000
wm.update_config(CNVpipeline)
submission_id = wm.create_submission(CNVpipeline['name'],etype='sample_set',entity=samplesetname,expression='this.samples')
terra.wait_for_submission(submission_id)

In [ ]:
aggregate = wm.get_config('Aggregate_CN_seg_files')
aggregate

In [ ]:
wm.update_pair_set(samplesetname,val)

In [ ]:
aggregate['inputs']['aggregate_CN_segments_wrkflw.aggregate_CN_segments.sample_seg_files'] = 'this.pair.called_copy_ratio_segments_tumor'
wm.update_config(aggregate)
submission_id = wm.create_submission(config = aggregate['name'], entity=samplesetname)
terra.wait_for_submission(submission_id)

In [ ]:
aggregated = wm.get_entities(etype="pair_CNV_set")["combined_seg_file"].values[1]
aggregated

In [ ]:
! gsutil cp $aggregated temp/cn.aggregated.called.seg

you can even add `-i df -w 5 -h 5 --units in -r 200`

In [ ]:
%%R 
source("gkugener/RScripts/load_libraries_and_annotations.R")

In [ ]:
%%R
source("../ccle_processing/CCLE_postp_function.R")

In [ ]:
%%R 
a <- interpolate_gaps_in_segmented(process_segments(
    "temp/cn.aggregated.called.seg"))
segments <- extend_ends_of_segments(a$segs)

In [ ]:
%%R
segments[170:200,]

In [ ]:
%%R
segments %<>% dplyr::mutate(Source='st_jude')

In [ ]:
%%R 
write.table(segments, file = "temp/wgs.st.jude.segmented.cn", sep = ',', quote = F, row.names = F) 
# What we upload to taiga

In [ ]:
%%R
allENTREZG <- generate_entrez_genes()

In [ ]:
%%R
segments %<>% mutate(seqnames=gsub('chr', '', seqnames)) 
head(segments)


In [ ]:
%%R
data <- generate_gene_level_matrix_from_segments(allENTREZG, segments)

In [ ]:
%%R
head(data$gene_level_data_hg38)

# Validate the results

In [ ]:
%%R
write.table(data$gene_level_data_hg38, file = "temp/wgs.st.jude.gene.cn", sep = ',', quote = F, row.names = T)

In [ ]:
genecn = pd.read_csv("temp/wgs.st.jude.gene.cn", sep = ',')

In [ ]:
segmentcn_wes = tc.get(name='copy-number-d4d9', version=5, file='wes.st.jude.segmented.cn')
segmentcn_wgs = pd.read_csv("temp/wgs.st.jude.segmented.cn", sep = ',')

In [ ]:
ccle.checkAmountOfSegments(segmentcn_wgs,thresh = 850)

In [ ]:
ccle.checkGeneChangeAccrossAll(genecn, thresh=1.5)

In [ ]:
segmentcn_wgs

In [ ]:
ccle.checkDifferencesWESWGS(segmentcn_wes, segmentcn_wgs)

In [ ]:
plots = wm.get_entities(etype="pair_CNV")["modeled_segments_plot_tumor"].values
for plot in plots:
    ! gsutil cp $plot temp/

In [ ]:
for plot in plots:
    display(Image('temp/'+plot.split('/')[-1]))

We are interested in seeing if there is 18q/VPS4B loss in RMS. We previously attempted to look at CN loss in an RMS dataset that used SOLID sequencing. However, our results were inconclusive due to problems with the quality of the data. We expected the data from St Jude's to be of much higher quality and that we should get a better sense of the CN profile of these tumors from this dataset.

### TODOS:

* Check the seg files in IGV look for hypo diploid samples with large gains/losses
* Before plotting - check on the sample metadata and remove some samples that are PDXs etc...
* Possibly run ABSOLUTE to get ploidy and whole genome doubling calls
* Summary figure of heatmap on 18q/SMAD4 sorted by deepest deletions

In [ ]:
%%R
st.jude.sample.info <- load.from.taiga(data.name='sample-annotations-7ca1', data.version=2, data.file='sample.info')

# Load the St Jude WES and WGS copy number data
st.jude.provided.bed.gene.cn <- load.from.taiga(data.name='copy-number-d4d9', data.version=6, data.file='st.jude.provided.bed.gene.cn')
st.jude.provided.bed.segmented.cn <- load.from.taiga(data.name='copy-number-d4d9', data.version=6, data.file='st.jude.provided.bed.segmented.cn')
wes.st.jude.gene.cn <- load.from.taiga(data.name='copy-number-d4d9', data.version=6, data.file='wes.st.jude.gene.cn')
wes.st.jude.segmented.cn <- load.from.taiga(data.name='copy-number-d4d9', data.version=6, data.file='wes.st.jude.segmented.cn')
wgs.st.jude.gene <- load.from.taiga(data.name='copy-number-d4d9', data.version=6, data.file='wgs.st.jude.gene')
wgs.st.jude.segmented <- load.from.taiga(data.name='copy-number-d4d9', data.version=6, data.file='wgs.st.jude.segmented')

cyto_band_file <- '~/Documents/Analysis/RScripts/Common_annotations/hg38_cytoband.gz'

In [ ]:
%%R 
total_segments_per_sample <- rbind(
  wgs.st.jude.segmented %>%
    dplyr::rename(Sample=DepMap_ID) %>%
    group_by(Sample) %>%
    dplyr::summarise(count=n()) %>%
    mutate(Source='WGS'),
  wes.st.jude.segmented.cn %>%
    group_by(Sample) %>%
    dplyr::summarise(count=n()) %>%
    mutate(Source='WES'),
  st.jude.provided.bed.segmented.cn %>%
    group_by(Sample) %>%
    dplyr::summarise(count=n()) %>%
    mutate(Source='BED')
)

In [ ]:
%%R
chr_cutoffs <- generate_chromosome_cutoffs_list(cyto_band_file = cyto_band_file)
columns_for_segment_plotting <- c('Sample', 'Chromosome', 'Start', 'End', 'Segment_Mean', 'Source')

In [ ]:
%%R
# The plots below give us a sense of the copy number profile of the samples 
continuous.wes.st.jude.segmented.cn <- wes.st.jude.segmented.cn %>%
  mutate(Start=Start+chr_cutoffs[paste0('chr', gsub('^chr', '', Chromosome))], End=End+chr_cutoffs[paste0('chr', gsub('^chr', '', Chromosome))]) %>%
  mutate(Source='WES') %>%
  dplyr::select(columns_for_segment_plotting)
continuous.wgs.st.jude.segmented.cn <- wgs.st.jude.segmented %>%
  mutate(Start=start+chr_cutoffs[paste0('chr', gsub('^chr', '', seqnames))], End=end+chr_cutoffs[paste0('chr', gsub('^chr', '', seqnames))]) %>%
  mutate(Source='WGS') %>%
  dplyr::rename(Sample=DepMap_ID, Chromosome=seqnames) %>%
  dplyr::select(columns_for_segment_plotting)
continuous.provided.bed.st.jude.segmented.cn <- st.jude.provided.bed.segmented.cn %>%
  mutate(Start=Start+chr_cutoffs[paste0('chr', gsub('^chr', '', Chromosome))], End=End+chr_cutoffs[paste0('chr', gsub('^chr', '', Chromosome))]) %>%
  dplyr::select(columns_for_segment_plotting)

combined_segments_all_sources <- rbind(continuous.wes.st.jude.segmented.cn, continuous.wgs.st.jude.segmented.cn, continuous.provided.bed.st.jude.segmented.cn)

# Plot the data
# (1) Filter out all the germline samples
tumor_samples_only <- st.jude.sample.info %>% filter(sample_type != 'Germline') %$% sample_name
tumor_segments_only <- combined_segments_all_sources %>% filter(Sample %in% tumor_samples_only)

sample_order_for_plot <- unique(tumor_segments_only$Sample)

# Plot
tumor_segments_only %<>% mutate(CN_capped=ifelse(log2(Segment_Mean) > 1, 1, ifelse(log2(Segment_Mean) < -1, -1, log2(Segment_Mean))))
tumor_segments_only %<>% mutate(Sample=factor(Sample, levels=sample_order_for_plot))
sample_labels <- setNames(seq(1, length(sample_order_for_plot)), sample_order_for_plot)

continuous_chromosome_plots_all <- ggplot(tumor_segments_only) +
  geom_rect(aes(xmin=Start, xmax=End, ymin=as.integer(Sample)-0.5, ymax=as.integer(Sample)+0.5, fill=CN_capped)) +
  scale_fill_gradient2(low='blue', high='red', mid='white', midpoint=0) +
  scale_x_continuous(breaks=chr_cutoffs, labels=names(chr_cutoffs)) +
  scale_y_continuous(breaks=sample_labels, labels=names(sample_labels)) +
  facet_wrap(~Source, ncol = 1) +
  theme(
    axis.text.x = element_text(angle=90, hjust=1, vjust=0.5, size=10),
    axis.text.y = element_text(size=10)
  )
ggsave(continuous_chromosome_plots_all, filename = file.path(plot_saving_directory, 'temp/continuous_segments_complete_all_source.pdf'), width=10, height = 30)

# Upload the validated results

In [ ]:

tc = TaigaClient()
tc.update_dataset(dataset_permaname="copy-number-d4d9", 
                  upload_file_path_dict={'temp/wgs.st.jude.gene.cn': 'NumericMatrixCSV',
                                        'temp/wgs.st.jude.segmented.cn': 'TableCSV'},
                 dataset_description="updating to")